In [1]:
import cv2

In [12]:
import tensorflow as tf
import numpy as np
#import opencv as cv2
import os
#from PIL import Image
from tensorflow.keras.applications import ResNet50

# Load the DeepLabV3 model pre-trained on the COCO dataset
model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False)

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            images.append((filename, img))
    return images

def segment_humans(image):
    h, w, _ = image.shape
    input_image = cv2.resize(image, (224, 224))
    input_image = tf.keras.applications.resnet50.preprocess_input(input_image)
    input_image = np.expand_dims(input_image, axis=0)

    # Get the segmentation mask
    preds = model.predict(input_image)
    preds = np.argmax(preds.squeeze(), axis=-1)

    # Resize the mask to the original image size
    preds = cv2.resize(preds, (w, h), interpolation=cv2.INTER_NEAREST)

    # Create a binary mask where the human is located
    mask = preds == 1  # Class ID 15 corresponds to humans in COCO dataset

    return mask

def save_human_cutouts(images, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename, image in images:
        mask = segment_humans(image)
        cv2.imshow("Image", mask.astype(np.uint8))
        cv2.waitKey()
        cv2.destroyAllWindows()
        human_cutout = cv2.bitwise_and(image, image, mask=mask.astype(np.uint8))

        # Save the cutout
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, human_cutout)

input_folder = 'INPUT FOLDER'
output_folder = input_folder +'\out'

images = load_images_from_folder(input_folder)
save_human_cutouts(images, output_folder)


In [23]:
import tensorflow as tf
import numpy as np
import cv2
import os
from tensorflow.keras.applications import ResNet50

# Load the DeepLabV3 model
MODEL_NAME = 'deeplabv3_mnv2_pascal_train_aug'
_DOWNLOAD_URL_PREFIX = "http://download.tensorflow.org/models/"
_MODEL_URLS = {
    'deeplabv3_mnv2_pascal_train_aug': 'deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz',
}
_TARBALL_NAME = 'deeplab_model.tar.gz'

model_dir = tf.keras.utils.get_file(
    fname=_TARBALL_NAME,
    origin=_DOWNLOAD_URL_PREFIX + _MODEL_URLS[MODEL_NAME],
    untar=True)

model_path = os.path.join(model_dir, "frozen_inference_graph.pb")

def load_model(model_path):
    graph = tf.Graph()
    with graph.as_default():
        graph_def = tf.compat.v1.GraphDef()
        with tf.io.gfile.GFile(model_path, "rb") as f:
            graph_def.ParseFromString(f.read())
            tf.import_graph_def(graph_def, name="")
    return graph

def segment_humans(image, graph):
    input_size = 513
    width, height = image.shape[1], image.shape[0]
    resized_image = cv2.resize(image, (input_size, input_size))
    resized_image = np.asarray(resized_image, dtype=np.uint8)

    with tf.compat.v1.Session(graph=graph) as sess:
        input_tensor = graph.get_tensor_by_name('ImageTensor:0')
        output_tensor = graph.get_tensor_by_name('SemanticPredictions:0')
        predictions = sess.run(output_tensor, feed_dict={input_tensor: [resized_image]})
        predictions = predictions[0]

    mask = cv2.resize(predictions.astype(np.uint8), (width, height), interpolation=cv2.INTER_CUBIC)

    # Create binary mask for the human class
    human_mask = (mask == 15).astype(np.uint8)
    return human_mask

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            images.append((filename, img))
    return images

def save_human_cutouts(images, output_folder, graph):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename, image in images:
        mask = segment_humans(image, graph)
        alpha_channel = mask * 255

        # Create a 4-channel image with transparency
        b, g, r = cv2.split(image)
        rgba = [b, g, r, alpha_channel]
        human_cutout = cv2.merge(rgba)

        # Save the cutout as a PNG with transparency
        output_path = os.path.join(output_folder, os.path.splitext(filename)[0] + '.png')
        cv2.imwrite(output_path, human_cutout)
        
input_folder = 'INPUT FOLDER'
output_folder =  input_folder +'\out'

graph = load_model(model_path)
images = load_images_from_folder(input_folder)
save_human_cutouts(images, output_folder, graph)